<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Concentration_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Concentration calculation
This notebook is used to calculate the concentration of phtalates in an environemental sample using an internal standard for quantitation. The Calibration_calculation.ipynb notebook, yields severall of the required files nescessary for the calculations in this notebook. The files *Cal_{n}_analyte_linreg.csv* need to be used which is located in the *calibration_output* folder on the google drive. If this notebook is not applied from my machine in connection with my google drive, either the path to the required files needs to be adabted or changed.
The following code allows for the calculation of the concentration, by using the linear regression form a calibration series and the ratio of the signal of the analyte with the internal standard (area). All the data regarding the calibration are present on my google drive in the folders: *calibration_input* and *calibration_output*, while a new google drive folder will be used to save the final data in as well as a new input folder.

*Google drive folder:*
quantitation_input
quantitation_output

The .csv files containing the peak list from the sample should be saved in the in the input folder. The names should follow a consequent naming sceme:

***Sample_{date_of_analysis}_{tag_bonus_info}_{Sample_Nr}_pl.csv***

Here the Sample number *(Sample_Nr)* is of key importance and needs to be properly documented in the Labjournal, as it is the main way of differentiation.


The following code is used to import all the nescessary libraries and mount the google drive.

In [1]:
import numpy as np
from google.colab import drive
import glob
import pandas as pd
import os
import csv
from datetime import datetime
import re

drive.mount('/content/drive', force_remount = True)

sample_nr_low = 1 #needs to be addapted to 1 nr in the series
sample_nr_high = 2 #needs to be addapted to the last nr of the series

Mounted at /content/drive


The following code is the addapted code from the Calibration_calculation.ipynb and yields the separated in separated .csv files, which will be used to calculate the ratio in a following code segment.

The output file has the following filename and is saved in the quantitation_output folder:

***Sample_{n}_row{i+1}.csv***

In [7]:
#Loop over all the designated sample files.
for n in range(sample_nr_low, sample_nr_high):
    files = glob.glob(f'/content/drive/My Drive/quantitation_input/*_{n}_pl.csv')

    if not files:
        print(f'Files matching pattern Sample Nr. {n} was not found.')
        continue

    else:
        # Loop over each row and save a separate data frame for each
        df = pd.read_csv(files[0], header=4)
        num_rows = len(df)

        for i in range(num_rows):
            row_dfs = []
            for file in files:
                df = pd.read_csv(file, header=4, na_values='')

                # Extract the i'th row of the CSV file and append it to the row data frame list
                row = df.iloc[i, :].dropna().astype(str)
                row_dfs.append(row)

            # Concatenate the row data frames along the columns axis and convert to a data frame
            combined_df = pd.concat(row_dfs, axis=1).T

            # Save the combined data frame to a CSV file with a name indicating the row number
            combined_df.to_csv(f'/content/drive/My Drive/quantitation_output/Sample_{n}_row{i+1}.csv', index=False, header=False)

            # Convert all values to strings before joining and print the row in the desired format
            formatted_row = '\t'.join(map(str, combined_df.values[0]))
            print(formatted_row)

14.7	14.64	14.77	5489681148.639	100.0	2656951262.793	100.0
17.48	17.42	17.54	6469844114.261	59.84	3289498112.0	63.61
20.79	20.74	20.87	12943079734.584	100.0	6034264750.996	100.0
23.92	23.87	24.03	3628155886.189	33.56	1645675970.227	31.82
29.63	29.58	29.9	498666324.815	4.61	165925032.257	3.21
32.17	32.12	32.41	124651834.463	1.15	40633104.946	0.79
34.55	34.5	34.76	90363929.442	0.84	29846969.497	0.58


The following code allows to calculate the ratio of the analytes with the internal standard. The output is saved as:

***Sample_{n}_row*_ratio.csv**

In [8]:
# Loop over all the designated sample files. Currently works but i am not shure if it might be just exactly the same as the first one, it works but not iterative...
for n in range(sample_nr_low, sample_nr_high):
    divisor_file = f'/content/drive/My Drive/quantitation_output/Sample_{n}_row3.csv'
    directory = '/content/drive/My Drive/quantitation_output'

    if not os.path.isfile(divisor_file):
        print(f"Divisor file does not exist: {divisor_file}")
        continue  # Skip to the next iteration if the divisor file is missing

    divisor_row = pd.read_csv(divisor_file, sep=',', header=None)
    csv_files = glob.glob(os.path.join(directory, f'*_row*.csv'))

    # Process each CSV file
    for csv_file in csv_files:
        if csv_file == divisor_file or '_ratio' in csv_file:
            continue  # Skip the divisor file itself and ratio files
        data_df = pd.read_csv(csv_file, sep=',', header=None)
        data_df = data_df.apply(pd.to_numeric, errors='coerce')  # Convert values to numeric

        # Perform division on the data
        divided_data_df = data_df.div(divisor_row, axis='columns')

        # Create a new file name with the "_ratio" tag
        new_file_name = os.path.splitext(csv_file)[0] + '_ratio.csv'

        divided_data_df.to_csv(new_file_name, sep=',', header=False, index=False)
        control_import_df = pd.read_csv(new_file_name, sep=',', header=None)

        # Compare the recalculated values with the content of the ratio file
        if control_import_df.equals(divided_data_df):
            print(f"The files match: {new_file_name}")
        else:
            print(f"The files do not match: {new_file_name}")

        print(control_import_df)
        print(divided_data_df)
        print('****************************************************')

The files do not match: /content/drive/My Drive/quantitation_output/Sample_1_row1_ratio.csv
          0         1         2        3    4         5    6
0  0.707071  0.705882  0.707714  0.42414  1.0  0.440311  1.0
          0         1         2        3    4         5    6
0  0.707071  0.705882  0.707714  0.42414  1.0  0.440311  1.0
****************************************************
The files match: /content/drive/My Drive/quantitation_output/Sample_1_row2_ratio.csv
          0         1         2         3       4         5       6
0  0.840789  0.839923  0.840441  0.499869  0.5984  0.545137  0.6361
          0         1         2         3       4         5       6
0  0.840789  0.839923  0.840441  0.499869  0.5984  0.545137  0.6361
****************************************************
The files do not match: /content/drive/My Drive/quantitation_output/Sample_1_row4_ratio.csv
          0         1         2         3       4         5       6
0  1.150553  1.150916  1.151414  0.280316

The following code takes the ratio files as well as the parameters for the linear regression (analyte specific), and yields a final output file that contains all the information, such as the information of the ratio used for the quantitaiton, the parameters for the linear equation as well as other information such as compound id and other general sample information.

The final outpur files have the following name format:

***Sample_{n}_quantitation_output.csv***


In [9]:

current_date = datetime.today().strftime('%Y-%m-%d')
output_dir = '/content/drive/My Drive/quantitation_export/'
#header_row1 = [f'Analysis Nr. {n}', 'Phthalate concentration calculation', 'Concentration_calibration.ipynb', 'Till Vollmer', current_date]
header_row2 = ['Analyte', 'Intercept', 'Slope', 'Value', 'Concentration [microg/ml]']

row_descriptors = ['place holder','Dimethylphthalate', 'Diethylphthalate', 'Internal standard', 'Dibutylphthalate', 'Benzylbutylphthalate', 'Bis(2-ethyl-hexyl)phthalate', 'Di-n-octylphthalate']
file_pattern = '/content/drive/My Drive/quantitation_output/Sample_{}_row{}_ratio.csv'

for n in range(sample_nr_low, sample_nr_high):
    output_file = output_dir + f'Sample_{n}_concentration_output.csv'
    header_row1 = [f'Analysis Nr. {n}', 'Phthalate concentration calculation', 'Concentration_calibration.ipynb', 'Till Vollmer', current_date]

    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header_row1)
        writer.writerow(header_row2)

        for i in range(1, 8):
          if i == 3:
              continue
          print(f'Sample nr: {n}')
          print(f'Compound iteration: {i}')
          linreg_df = pd.read_csv(f'/content/drive/My Drive/calibration_export/Cal_{i}_analyte_linreg.csv')
          print(f'linear function information: {linreg_df}')
          descriptor = row_descriptors[i]
          print(f'Descriptor: {descriptor}')
          file_path = file_pattern.format(n, i)
          if not os.path.isfile(file_path):
            print(f"File not found for Sample {n}, Row {i}")
            continue

          sample_df = pd.read_csv(file_pattern.format(n, i), header=None, delimiter=',')  # Corrected line
          print(f'Sample dataframe: {sample_df}')
          intercept_str = linreg_df['Intercept'].values[0]
          slope_str = linreg_df['Slope'].values[0]
          intercept = float(re.findall(r'[-\d.]+', intercept_str)[0])
          slope = float(re.findall(r'[-\d.]+', slope_str)[0])
          fourth_column = sample_df.iloc[:, 3]  # Accessing the 4th column
          value = float(fourth_column.iloc[0])  # Assuming you want the value from the first row (index 0)
          concentration = slope * value + intercept
          print(f"Calculated concentration: {concentration}")
          row_data = [descriptor, intercept, slope, value, concentration]
          writer.writerow(row_data)

          # Additional print statements for debugging
          print(f"Sample DataFrame shape: {sample_df.shape}")
          print(f"Linreg DataFrame shape: {linreg_df.shape if linreg_df is not None else None}")
          print("------------------------------------------------------------------------")


Sample nr: 1
Compound iteration: 1
linear function information:        Intercept            Slope  R2 Value
0  [-0.01726225]  [[16.48584551]]  0.967163
Descriptor: Dimethylphthalate
Sample dataframe:           0         1         2        3    4         5    6
0  0.707071  0.705882  0.707714  0.42414  1.0  0.440311  1.0
Calculated concentration: 6.975048480636993
Sample DataFrame shape: (1, 7)
Linreg DataFrame shape: (1, 3)
------------------------------------------------------------------------
Sample nr: 1
Compound iteration: 2
linear function information:        Intercept            Slope  R2 Value
0  [-0.00909398]  [[13.87985743]]  0.978623
Descriptor: Diethylphthalate
Sample dataframe:           0         1         2         3       4         5       6
0  0.840789  0.839923  0.840441  0.499869  0.5984  0.545137  0.6361
Calculated concentration: 6.929016248921768
Sample DataFrame shape: (1, 7)
Linreg DataFrame shape: (1, 3)
----------------------------------------------------------